# Scikit-Learn Iris Model using customData

* Wrap a scikit-learn python model for use as a prediction microservice in seldon-core
    * Run locally on Docker to test
    * Deploy on seldon-core running on a Kubernetes cluster

## Dependencies

* [s2i](https://github.com/openshift/source-to-image)
* Seldon Core v1.0.3+ installed
* `pip install sklearn seldon-core protobuf grpcio`

In [47]:
!pip3 install sklearn seldon-core protobuf grpcio 
!pip3 install grpcio-tools
!pip3 install --upgrade protobuf
!pip3 install --upgrade pip

  Using cached protobuf-3.20.3-cp39-cp39-macosx_10_9_x86_64.whl (982 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.12
    Uninstalling protobuf-4.21.12:
      Successfully uninstalled protobuf-4.21.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch2tflite 1.0.0 requires tflite-runtime~=2.5, which is not installed.
tensorflow 2.11.0 requires flatbuffers>=2.0, but you have flatbuffers 1.12 which is incompatible.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorflow-serving-api 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
grpcio-tools 1.51.1 requires protobuf<5.0dev,>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
torch2tflite 1.0.0 requires torch==1.8.1, but you have torch 1.9.1 which is incompatible.
on

In [53]:
# !pip3 uninstall protobuf
!pip3 install protobuf==4.21

Found existing installation: protobuf 4.21.12
Uninstalling protobuf-4.21.12:
  Would remove:
    /Users/dileep.gadiraju/opt/anaconda3/lib/python3.9/site-packages/google/_upb/_message.abi3.so
    /Users/dileep.gadiraju/opt/anaconda3/lib/python3.9/site-packages/google/protobuf/*
    /Users/dileep.gadiraju/opt/anaconda3/lib/python3.9/site-packages/protobuf-4.21.12.dist-info/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.0/483.0 kB 4.0 MB/s eta 0:00:0000:0100:01
Reason for being yanked: Required python version not configured correctly (https://github.com/protocolbuffers/protobuf/issues/10076)


## Train locally

In [48]:
import os

import numpy as np
from sklearn import datasets
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline


def main():
    clf = LogisticRegression()
    p = Pipeline([("clf", clf)])
    print("Training model...")
    p.fit(X, y)
    print("Model trained!")

    filename_p = "./custom-models/iris/IrisClassifier.sav"
    print("Saving model in %s" % filename_p)
    joblib.dump(p, filename_p)
    print("Model saved!")


if __name__ == "__main__":
    print("Loading iris data set...")
    iris = datasets.load_iris()
    X, y = iris.data, iris.target
    print("Dataset loaded!")
    main()

Loading iris data set...
Dataset loaded!
Training model...
Model trained!
Saving model in ./custom-models/iris/IrisClassifier.sav
Model saved!


/Users/dileep.gadiraju/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Custom Protobuf Specification

First, we'll need to define our custom protobuf specification so that it can be leveraged.

In [49]:
%%writefile ./custom-models/iris/iris.proto

syntax = "proto3";

package iris;

message IrisPredictRequest {
    float sepal_length = 1;
    float sepal_width = 2;
    float petal_length = 3;
    float petal_width = 4;
}

message IrisPredictResponse {
    float setosa = 1;
    float versicolor = 2;
    float virginica = 3;
}

Overwriting ./custom-models/iris/iris.proto


## Custom Protobuf Compilation

We will need to compile our custom protobuf for python so that we can unpack the `customData` field passed to our `predict` method later on.

In [50]:
!python3 -m grpc.tools.protoc --python_out=./custom-models/iris --proto_path=./custom-models/iris iris.proto

## gRPC test

Wrap model using s2i

In [51]:
!s2i build ./custom-models/iris seldonio/seldon-core-s2i-python37-ubi8:1.7.0-dev seldonio/sklearn-iris-customdata:0.1 --env MODEL_NAME="iris_pb2.py" --env SERVICE_TYPE="MODEL" --env PERSISTENCE=0

error: Unable to load docker config: json: cannot unmarshal string into Go value of type docker.dockerConfig
---> Installing application source...
Created wheel for PTable: filename=PTable-0.9.2-py3-none-any.whl size=22907 sha256=2298be0ca09b501316053819a12b0c02e426675d37094f0178a6ef9b90b11fed
Stored in directory: /root/.cache/pip/wheels/33/df/2f/674985b3f8a2de3f96357d1eadef5110f74fa91b3785e52a54
Successfully built PTable
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
created path: ./licenses/license_info.csv
created path: ./licenses/license.txt
Build completed successfully


Serve the model locally

In [38]:
!docker run --name "iris_predictor" -d --rm -p 5000:5000 seldonio/sklearn-iris-customdata:0.1

b98aeeb76c816c8db54e666b7892e5aae27b351d314678d54799d258b8d1934c


Test using custom protobuf payload

In [ ]:
import grpc
from iris_pb2 import IrisPredictRequest, IrisPredictResponse

from seldon_core.proto import prediction_pb2, prediction_pb2_grpc

channel = grpc.insecure_channel("localhost:5000")
stub = prediction_pb2_grpc.ModelStub(channel)

iris_request = IrisPredictRequest(
    sepal_length=7.233, sepal_width=4.652, petal_length=7.39, petal_width=0.324
)

seldon_request = prediction_pb2.SeldonMessage()
seldon_request.customData.Pack(iris_request)

response = stub.Predict(seldon_request)

iris_response = IrisPredictResponse()
response.customData.Unpack(iris_response)

print(iris_response)

Stop serving model

In [ ]:
!docker rm iris_predictor --force

## Setup Seldon Core

Use the [setup notebook](https://github.com/SeldonIO/seldon-core/blob/master/notebooks/seldon_core_setup.ipynb) to setup Seldon Core with an ingress - either Ambassador or Istio

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:

* Ambassador: `kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080`
* Istio: `kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8003:80`

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

## Deploy your Seldon Model

We first create a configuration file:

In [ ]:
%%writefile sklearn_iris_customdata_deployment.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-deployment-example
spec:
  name: sklearn-iris-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: groszewn/sklearn-iris-customdata:0.1
          imagePullPolicy: IfNotPresent
          name: sklearn-iris-classifier
    graph:
      children: []
      endpoint:
        type: GRPC
      name: sklearn-iris-classifier
      type: MODEL
    name: sklearn-iris-predictor
    replicas: 1

### Run the model in our cluster

Apply the Seldon Deployment configuration file we just created

In [ ]:
!kubectl create -f sklearn_iris_customdata_deployment.yaml

### Check that the model has been deployed

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=seldon-deployment-example -o jsonpath='{.items[0].metadata.name}')

## Test by sending prediction calls

`IrisPredictRequest` sent via the `customData` field.

In [ ]:
iris_request = IrisPredictRequest(
    sepal_length=7.233, sepal_width=4.652, petal_length=7.39, petal_width=0.324
)

seldon_request = prediction_pb2.SeldonMessage()
seldon_request.customData.Pack(iris_request)

channel = grpc.insecure_channel("localhost:8003")
stub = prediction_pb2_grpc.SeldonStub(channel)

metadata = [("seldon", "seldon-deployment-example"), ("namespace", "seldon")]

response = stub.Predict(request=seldon_request, metadata=metadata)

iris_response = IrisPredictResponse()
response.customData.Unpack(iris_response)

print(iris_response)

### Cleanup our deployment

In [ ]:
!kubectl delete -f sklearn_iris_customdata_deployment.yaml